HW

 **Transformers. Text summarization**

**Task:**

Write a system to generate annotations using HuggingFace Transformers

In [11]:
from transformers import pipeline
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#facebook/bart-large-cnn model

In [3]:
#define summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
#Article to summarize


ARTICLE = """ Alice's Adventures in Wonderland (also known as Alice in Wonderland)
is an 1865 English children's novel by Lewis Carroll, a mathematics don at the
University of Oxford. It details the story of a girl named Alice who falls through a
rabbit hole into a fantasy world of anthropomorphic creatures. It is seen as an example
of the literary nonsense genre. The artist John Tenniel provided 42 wood-engraved illustrations for the book.

It received positive reviews upon release and is now one of the best-known works
of Victorian literature; its narrative, structure, characters and imagery have had
a widespread influence on popular culture and literature, especially in the fantasy
genre.[1][2] It is credited as helping end an era of didacticism in children's literature,
inaugurating an era in which writing for children aimed to "delight or entertain".[3] The tale
plays with logic, giving the story lasting popularity with adults as well as with children.[4]
The titular character Alice shares her name with Alice Liddell, a girl Carroll knew—scholars disagree
about the extent to which the character was based upon her.[5][6]

The book has never been out of print and has been translated into 174 languages.
Its legacy includes adaptations to screen, radio, visual art, ballet, opera, and musical
theatre, as well as theme parks, board games and video games.[7] Carroll published a sequel in
1871 entitled Through the Looking-Glass and a shortened version for young children, The Nursery
"Alice", in 1890.
"""

In [9]:
#output

print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': "The Adventures of Alice in Wonderland was published in 1871. It is one of the most popular children's books of all time. It has been adapted for film, television, radio, and the stage."}]


#csebuetnlp/mT5_multilingual_XLSum model (trying out ukrainian lamguage)

In [ ]:
#pattern re whitespace

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [23]:
#article in original language

article_text_ukr = """Він, швидко поробивши човни,
На синє море поспускав,
Троянців насажавши повні,
І куди очі почухрав.
Но зла Юнона, суча дочка,
Розкудкудакалась, як квочка, —
Енея не любила — страх;
Давно уже вона хотіла,
Його щоб душка полетіла
К чортам і щоб і дух не пах.
3 Еней був тяжко не по серцю
Юноні — все її гнівив;
Здававсь гірчійший їй від перцю,
Ні в чім Юнони не просив;
Но гірш за те їй не любився,
Що, бачиш, в Трої народився
І мамою Венеру звав;
І що його покійний дядько,
Парис, Пріамове дитятко,
Путивочку Венері дав."""


In [25]:
# translated article
article_text_en = """
Constructing his boats at great speed,
He launched them on the bluish sea
And filled them with the men he'd need
As he sailed toward his destiny.
But Juno, daughter of a bitch,
A cackling hen with fighting itch,
Loathed him for being proud and deft.
She wished to see that his soul would
Fly to the deuce for good,
And no trace of him would be left.

To her he was like pepper, full of gall,
He always irritated her.
Her heart couldn't stomach him at all.
He was not her soliciting idolater.
She loathed him and it would annoy
Her so for he was born in Troy.
Besides, that Paris, his dad's brother,
Bestowed the apple as his recognition
Of triumph in a beauty competition
On Venus, his attractive mother
"""

In [ ]:
#model name
model_name = "csebuetnlp/mT5_multilingual_XLSum"

#define tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#define model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [21]:

def summary_func(article):
    # for this model we need  to define it input and output ids
    input_ids = tokenizer(
        [WHITESPACE_HANDLER(article)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]
    # decode tokenized topic
    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary

##test model

In [24]:
print(summary_func(article_text_ukr))

У квітні цього року був маленький чоловік.


In [26]:
print(summary_func(article_text_en))

When he was born in Troy, he always wanted to be proud and deft.


##compare with previous model

In [27]:
print(summary_func(ARTICLE))

One of the best-known Victorian children's novels has been released at the British Library.


#Trying out Ukrainian language again :)

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")

model = AutoModelForSeq2SeqLM.from_pretrained("SGaleshchuk/t5-large-ua-news")

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="pt")
##### Try on your example

summary = summarizer(article_text_ukr, min_length=3, max_length = 128)


In [31]:
print(summary)

[{'summary_text': 'Еней, який був тяжко не по серцю, і не любив свою сучу дочку, швидко повернувся на Трою, а її сучу дочку — Юнону, — вона не любила. '}]
